In [1]:
import pandas as pd
import numpy as np


In [31]:
data = pd.read_csv('combined-newsqa-data-v1.csv')
print(data.columns)
print(type(data['is_question_bad'][0]))

# remove Q/A pairs that are invalid or missing
#data = data[(data.is_question_bad=='0.0') & (data.is_answer_absent=='0.0')]
data.head()

Index(['story_id', 'question', 'answer_char_ranges', 'is_answer_absent',
       'is_question_bad', 'validated_answers', 'story_text'],
      dtype='object')
<class 'str'>


,story_id,question,answer_char_ranges,is_answer_absent,is_question_bad,validated_answers,story_text
0,./cnn/stories/42d01e187213e86f5fe617fe32e716ff...,What was the amount of children murdered?,294:297|None|None,0.0,0.0,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort..."
1,./cnn/stories/c48228a52f26aca65c31fad273e66164...,Where was one employee killed?,34:60|1610:1618|34:60,0.0,0.0,NaN,(CNN) -- Fighting in the volatile Sudanese reg...
2,./cnn/stories/c65ed85800e4535f4bbbfa2c34d7d963...,who did say South Africa did not issue a visa ...,103:127|114:127|839:853,0.0,0.0,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...
3,./cnn/stories/0cf66b646e9b32076513c050edf32a79...,How many years old was the businessman?,538:550|538:550,0.0,0.0,NaN,(CNN) -- England international footballer Ste...
4,./cnn/stories/13012604e3203c18df09289dfedd14cd...,What frightened the families?,690:742|688:791|630:646,0.0,0.0,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi..."


In [27]:
print(data['is_question_bad'][0])

0.0


In [7]:
# 119,633 Q/A's , 12088 articles
print(len(data))
print(len(data['story_text'].unique()))


119633
12088


In [9]:
first_doc = data['story_text'][0]
first_doc

'NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed "the house of horrors."\n\n\n\nMoninder Singh Pandher was sentenced to death by a lower court in February.\n\n\n\nThe teen was one of 19 victims -- children and young women -- in one of the most gruesome serial killings in India in recent years.\n\n\n\nThe Allahabad high court has acquitted Moninder Singh Pandher, his lawyer Sikandar B. Kochar told CNN.\n\n\n\nPandher and his domestic employee Surinder Koli were sentenced to death in February by a lower court for the rape and murder of the 14-year-old.\n\n\n\nThe high court upheld Koli\'s death sentence, Kochar said.\n\n\n\nThe two were arrested two years ago after body parts packed in plastic bags were found near their home in Noida, a New Delhi suburb. Their home was later dubbed a "house of horrors" by the Indian media.\n\n\n\nPandher was not named a main suspect by 

In [10]:
first_doc[294:297]

'19 '

In [11]:
second_doc = data['story_text'][2]
second_doc

'Johannesburg (CNN) -- Miffed by a visa delay that led the Dalai Lama to cancel a trip to South Africa, Archbishop Desmond Tutu lashed out at his government Tuesday, saying it had acted worse than apartheid regimes and had forgotten all that the nation stood for.\n\n\n\n\n\n"When we used to apply for passports under the apartheid government, we never knew until the last moment what their decision was," Tutu said at a news conference. "Our government is worse than the apartheid government because at least you were expecting it from the apartheid government.\n\n\n\n\n\n"I have to say that I can\'t believe this. I really can\'t believe this," Tutu said. "You have to wake me up and tell me this is actually happening here."\n\n\n\n\n\nThe Dalai Lama scrapped his planned trip to South Africa this week after the nation failed to issue him a visa in time, his spokesman said.\n\n\n\n\n\nVisa applications for him and his entourage were submitted to the South African High Commission in New Delhi,

In [13]:
print(second_doc[103:127])
print(second_doc[114:127])
print(second_doc[839:853])

Archbishop Desmond Tutu 
Desmond Tutu 
his spokesman 


Answers given by different human reviewers are in the answer_char_ranges column and the validated_answers column. These values are string index ranges within the document that represent the answer. For each row, we need to determine which character range is the best answer to use and extract it. There are a couple ways we could approach this problem, the first is to have a model that takes the question(str), the story(str), spits out an answer(str).

or we could have it spit back out the string indexes themselves. 

In [45]:
# Remove uneeded columns
df = data[['question','answer_char_ranges','validated_answers','story_text']]
df = df.iloc[0:100] # reduce size for development purposes
df.head()

,question,answer_char_ranges,validated_answers,story_text
0,What was the amount of children murdered?,294:297|None|None,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort..."
1,Where was one employee killed?,34:60|1610:1618|34:60,NaN,(CNN) -- Fighting in the volatile Sudanese reg...
2,who did say South Africa did not issue a visa ...,103:127|114:127|839:853,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...
3,How many years old was the businessman?,538:550|538:550,NaN,(CNN) -- England international footballer Ste...
4,What frightened the families?,690:742|688:791|630:646,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi..."


In [46]:
# loops through the rows and prints the question along with the first answer given
for i, row in df.iterrows():
    possible_ranges = row['answer_char_ranges']
    possible_ranges_list = possible_ranges.split('|')
    range1 = possible_ranges_list[0].split(':')
    print(range1)
    answer = row['story_text'][int(range1[0]):int(range1[1])]
    question = row['question']
    print(question)
    print(answer)
    break

['294', '297']
What was the amount of children murdered?
19 
